## Notebook C: Concatenating the transport routes

**TU Delft**<br>
**Author:** Ruth Nelson <br>

The main purpose of this example notebook is to illustrate the concatenation of the transportation lines with the edge network

1. Import libraries
2. Import dataframes
3. Calculate time-costs
4. Clean vertices and edges
5. Concatenate edges
6. Save dataframes to shp

## 1. Import libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import glob

In [ ]:
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', None)
#pd.set_option('display.notebook_repr_html', True)

## 2. Import the dataframes

In [ ]:
path_to_network = " "

In [ ]:
os.chdir(path_to_network)
for file in glob.glob("*"):
    print(file)

## Important: 

Prior to importing these edges I calculated the length for each edge in QGIS, I did it in QGIS because it is more accurate than Python taking into account the curvature of the earth, although you can also do it in Python. All the original streets have been broken up by the connecting vertices and edges. To calculate the lengths, one loads the edges data set into QGIS and then opens the attribute table and then the Field Calculater. In the Field Calculator, one clicks on Geometry and selects $length. The column needs to be named Length and it must be a real number. Press Ok and the new length variables are stored in a column called Length. Press the pencil in the top left hand corner to save the updates. Just to note, in order for this to work, the CRS of the edge file (which can be set in the layer properties) must be set to the CRS of the project in QGIS, which in the case of Cape Town is EPSG:3857.

In [ ]:
edges  = gpd.read_file('edges_b.shp') 

In [ ]:
edges

In [ ]:
vertices  = gpd.read_file('vertices_b.shp') 

In [ ]:
vertices

In [ ]:
path_to_transportation1_lines = " "

In [ ]:
os.chdir(path_to_transportation1_lines)
for file in glob.glob("*"):
    print(file)

In [ ]:
brt_edges  = gpd.read_file('Myciti_edges.shp') #load in brt edges

In [ ]:
path_to_transportation2_lines =" "

In [ ]:
brt_edges.plot()

In [ ]:
os.chdir(path_to_transportation2_lines)
for file in glob.glob("*"):
    print(file)

In [ ]:
railway_edges  = gpd.read_file('Railway_edges.shp') #load in railway edges

In [ ]:
railway_edges.plot()

## 3. Calculate time-cost of different edges

I will now calculate the time-cost of each edge based on the length of each street and the connector type

In [ ]:
#per minute an average human walks roughly 83.3m
walking = 83.3

In [ ]:
edges['time_cost'] = edges['Length']/walking

In [ ]:
edges['time_cost'] = edges['time_cost'].round(2)

In [ ]:
edges.head()

In [ ]:
#time-cost on a BRT connector is 0.5 min
edges.loc[edges['edge_type'].isin(['brt_connector']), 'time_cost'] = 0.5

In [ ]:
#time-cost on a Railway connector is 1 minute
edges.loc[edges['edge_type'].isin(['railway_connector']), 'time_cost'] = 1

In [ ]:
#time-cost on a land use connector is 0.5 minute
edges.loc[edges['edge_type'].isin(['land_use_connector']), 'time_cost'] = 0.5

##  4. Clean edges and vertices

In [ ]:
edges = edges[['edge_type','vertex_typ','id', 'from_id', 'to_id', 'Length','time_cost', 'geometry']]

In [ ]:
edges = edges.rename(columns ={'from_id':'source', 'to_id':'target'})

In [ ]:
#create an edge id for each Myciti edge
brt_edges['id'] = brt_edges['source'] + ', ' + brt_edges['target']

In [ ]:
#Create an edge and vertex type for each brt edge
brt_edges['edge_type'] = 'brt_line'
brt_edges['vertex_typ'] = 'not street'

In [ ]:
#create an edge id for each railway edge
railway_edges['id'] = railway_edges['source'] + ', ' + railway_edges['target']

In [ ]:
#Create an edge type for each railway edge
railway_edges['edge_type'] = 'railway_line'
railway_edges['vertex_typ'] = 'not street'

In [ ]:
railway_edges.head()

## 5. Concatenate Edges

In [ ]:
#concatenate road edges with brt edges
edges_f = pd.concat([edges, brt_edges])

In [ ]:
#concatenate edges_ff with railway edges
edges_ff = pd.concat([edges_f, railway_edges])

In [ ]:
#Reset index
edges_ff = edges_ff.reset_index().drop('index', axis = 1)

In [ ]:
edges_ff

## 6. Save

In [ ]:
path_to_save = " "

In [ ]:
os.chdir(path_to_save)
for file in glob.glob("*"):
    print(file)

In [ ]:
edges_ff.to_file('edges_c') #save edges

In [ ]:
vertices.to_file('vertices_c') #save vertices